In [ ]:
import torch
import matplotlib.pyplot as plt
import os

from analyze_pose_optimization import (
    plot_mean_error_per_class,
    plot_error_vs_transform_degrees,
    get_error_per_exp,
    get_euler_angles_per_transform
)

In [ ]:
path = os.path.expanduser("~/tbp/tbp.monty/projects/monty_runs/logs")
icp_path = os.path.join(path, "icp", "stats.pt")
icp_scipy_path = os.path.join(path, "icp_scipy", "stats.pt")
mcmc_path = os.path.join(path, "mcmc", "stats.pt")

In [ ]:
data = torch.load(icp_path)
sp_data = torch.load(icp_scipy_path)

In [ ]:
plot_mean_error_per_class(data)
plt.show()

In [ ]:
angles = get_euler_angles_per_transform(data)
all_errors = get_error_per_exp(data)
plot_error_vs_transform_degrees(all_errors, angles[:,0])
plt.show()

In [ ]:
exps = list(data.keys())
d0 = data[exps[0]]

In [ ]:
d0.keys()

In [ ]:
plt.plot(d0["time"])

In [ ]:
import numpy as np


def get_time_per_exp(data):

    n_exps = len(data.keys())
    keys = list(data.keys())
    n_samples_per_exp = len(data[keys[0]]["time"])
    all_times = np.zeros((n_exps, n_samples_per_exp))
    for i, k in enumerate(data.keys()):
        all_times[i] = data[k]["time"]
    
    return all_times

In [ ]:
all_times = get_time_per_exp(data)
all_times_sp = get_time_per_exp(sp_data)

In [ ]:
plt.imshow(all_times)

In [ ]:
print(f"mean runtime (icp standard): {np.mean(all_times)}")
print(f"std runtime (icp standard): {np.std(all_times)}")
print(f"mean runtime (scipy minimizer): {np.mean(all_times_sp)}")
print(f"std runtime (scipy minimizer): {np.std(all_times_sp)}")

In [ ]:
np.mean(all_times_sp) / np.mean(all_times)

In [ ]:
plot_mean_error_per_class(sp_data)
plt.show()

In [ ]:
sp_angles = get_euler_angles_per_transform(sp_data)
sp_all_errors = get_error_per_exp(sp_data)
plot_error_vs_transform_degrees(sp_all_errors, sp_angles[:,0])
plt.show()

In [ ]:
fig, ax = plt.subplots()

sp_errors = sp_all_errors.mean(axis=1)
sp_evars = sp_all_errors.var(axis=1)

errors = all_errors.mean(axis=1)
evars = all_errors.var(axis=1)

ax.errorbar(sp_angles[:, 0], sp_errors, yerr=sp_evars, marker=".", ls="none", label="scipy")
ax.errorbar(angles[:, 0], errors, yerr=evars, marker=".", ls="none", label="SVD")
ax.set_ylabel("Mean pointwise error")
ax.set_xlabel("Degrees of rotation")
plt.legend()

In [ ]:
fig, ax = plt.subplots()

sp_errors = sp_all_errors.mean(axis=1)
sp_evars = sp_all_errors.var(axis=1)

errors = all_errors.mean(axis=1)
evars = all_errors.var(axis=1)

ax.scatter(sp_angles[:, 0], sp_errors, marker=".", label="scipy")
ax.scatter(angles[:, 0], errors, marker=".", label="SVD")

ax.set_ylabel("Mean pointwise error")
ax.set_xlabel("Degrees of rotation")
plt.legend()

In [ ]:
n_exps = len(data.keys())
n_samples_per_exp = len(data["rotation_0"]["mean_pointwise_error"])
all_errors = np.zeros((n_exps, n_samples_per_exp))
for i, k in enumerate(data.keys()):
    all_errors[i] = data[k]["mean_pointwise_error"]

In [ ]:
all_deviations = np.array([data[k]["inverse_matrix_deviation"] for k in data.keys()])
rots = np.array([data[k]["transforms"].rotation.as_euler("xyz", degrees=True)[0] for k in data.keys()])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

for rot, err, dev in zip(rots, all_errors, all_deviations):
    ax[0].hist(err, 15, alpha=0.3, label="x_rotation=" + str(rot))
    ax[1].hist(dev, 15, alpha=0.3, label="x_rotation=" + str(rot))

ax[0].set_title("Pointwise error histogram")
ax[1].set_title("Deviation from identity matrix histogram")
plt.legend()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

errors = all_errors.mean(axis=1)
devs = np.mean(all_deviations, axis=1)

ax[0].plot(rots, errors, ".")
ax[0].set_ylabel("Mean pointwise error")
ax[0].set_xlabel("Degrees of x rotation")
ax[1].plot(rots, devs, ".")
ax[1].set_ylabel("sum(R^-1 hat R - I)^2")
ax[1].set_xlabel("Degrees of x rotation")


In [ ]:
print(np.corrcoef(errors, devs))

In [ ]:
ae = all_errors.ravel()
ad = all_deviations.ravel()
print(np.corrcoef(ae, ad))

In [ ]:
labels = np.array([data["rotation_1"]["labels"][i][0] for i in range(len(data["rotation_1"]["labels"]))])
unique_labels = np.unique(labels)
label_inds = dict()
for label in unique_labels:
    m = np.where(labels == label)[0]
    label_inds[label] = m

In [ ]:
# TODO this should be over all rotations
errors = np.array(data["rotation_1"]["mean_pointwise_error"])
label_to_error = dict()
label_to_error_var = dict()
for label in unique_labels:
    inds = label_inds[label]
    error = errors[inds].mean()
    label_to_error[label] = error
    label_to_error_var[label] = np.var(errors[inds])


In [ ]:
label_error = np.array([label_to_error[l] for l in unique_labels])
label_var = np.array([label_to_error_var[l] for l in unique_labels])

In [ ]:
fig, ax = plt.subplots()
ax.bar(x=unique_labels, height=label_error, yerr=label_var)
ax.set_xticklabels(unique_labels, rotation=70)

In [ ]:
fig, ax = plt.subplots(5, 2, figsize=(10, 8), sharey=True)
for i in range(n_exps):

    rot = rots[i]
    row = int(np.floor(i / 2))
    col = i % 2
    ax[row, col].plot(all_errors[i])
    ax[row, col].set_title("x_rotation=" + str(rot))
    # ax[1].plot(all_deviations[i], label="x_rotation=" + str(rot))

# ax[0].set_ylabel("Mean pointwise error")
# ax[1].set_ylabel("Identity deviation")
# plt.legend()
plt.tight_layout()

In [ ]:
disparities = [np.array(data[k]["angle_disparity"]) for k in data.keys()]
# data["rotation_0"]["angle_disparity"]

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
idx = 10

ax[0].plot(disparities[idx][:, 0] % 180, label="x-disparity")
# ax[0].plot(disparities[idx][:, 1], label="y-disparity")
# ax[0].plot(disparities[idx][:, 2], label="z-disparity")
ax[0].set_ylabel("Angle disparity")

ax[1].plot(all_errors[idx] * 100)
ax[1].set_ylabel("Error")
plt.legend()

In [ ]:
fig, ax = plt.subplots()
idx = 0

ax.plot(disparities[idx][:, 0] % 180, label="x-disparity")
ax.plot(all_errors[idx] * 100, label="error * 100")
# ax.set_ylabel("Error")
plt.legend()

In [ ]:
fig, ax = plt.subplots(5, 2, figsize=(10, 8), sharey=True)

cnt = 0
for i in range(5):
    for j in range(2):
        ax[i, j].plot(disparities[cnt][:, 0], label="x-disparity")
        ax[i, j].plot(all_errors[cnt] * 100, label="error * 100")
        r = data[f"rotation_{cnt}"]["transforms"].rotation.as_euler("xyz", degrees=True)[0]
        ax[i, j].plot(disparities[cnt][:, 0] + r, label="rotation + angle disparity")
        ax[i, j].axhline(180, linestyle="dashed", color="r", alpha=0.5)
        ax[i, j].axhline(-180, linestyle="dashed", color="r", alpha=0.5)
        ax[i, j].set_title(r)
        cnt += 1

plt.legend()
plt.tight_layout()



In [ ]:
data["rotation_4"]["transforms"].rotation.as_euler("xyz", degrees=True)

In [ ]:
np.corrcoef(all_errors.ravel(), d[:,:,0].ravel())

In [ ]:
d.shape

In [ ]:
np.random.normal(np.ones(3), np.ones(3), (3, 1))

In [ ]:
rpc = torch.randn(100, 3)
t = torch.normal(torch.ones(3), torch.ones(3))
rpct = rpc + t

In [ ]:
print(rpc[0,:])
print(t)
print(rpct[0,:])

In [ ]:
t = torch.normal(torch.ones(3), torch.ones(3))